# Equipe:
# Ana Cláudia Akemi Matsuki de Faria
# Danielle Bezerra Moreira
# Derek Freire Quaresma
# Henrique Sartori Siqueira
# Rafael Silva Barbon 

# Importando bibliotecas e dados

In [ ]:
#!pip install --upgrade pip
#!pip install -U coremltools
import coremltools
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import tensorflow as tf
import warnings #ignore warning messages
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.models import Sequential
from keras import backend as K
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

warnings.filterwarnings('ignore')

sns.set() # plot

In [ ]:
dataset = pd.read_csv("A_Z Handwritten Data.csv").astype('float32')
dataset.head()

Dataset: 
- Coluna 0 - label
- Demais colunas: pixels em tons de cinza
* X -> dados de treino
* Y -> resultado esperado (letra)

In [ ]:
dataset.rename(columns={'0':'label'}, inplace=True)
X = dataset.drop('label',axis = 1)
y = dataset['label']

# Visualizando o dataset

Percebe-se que o formato das letras para treino é de 28x28 (784), sendo que há 372450 letras dentro do dataset porém, nem todas possuem o formato 28x28 dentro do csv

In [ ]:
print("shape:",X.shape)
print("columns count:",len(X.iloc[1]))

X.head()

Mistura-se as letras em x para treino em x_shuffle e exibe 16 letras de exemplo

Index - letra correspondente (a=0...z=25)

In [ ]:
X_shuffle = shuffle(X)

plt.figure(figsize = (12,10))
row, colums = 4, 4
for i in range(16):  
    plt.subplot(colums, row, i+1)
    plt.imshow(X_shuffle.iloc[i].values.reshape(28,28),interpolation='nearest', cmap='Greys')
plt.show()
dataset['label']

In [ ]:
print("Estatística das letras do dataset")

# Ajustando os indexes para mapear as letras corretamente
alphabets_mapper = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X',24:'Y',25:'Z'} 
dataset_alphabets = dataset.copy()
dataset['label'] = dataset['label'].map(alphabets_mapper)

label_size = dataset.groupby('label').size()
label_size.plot.barh(figsize=(10,10))
plt.show()

print(label_size)

# Preparação dos dados
Divide os dados em dados de treino e teste

In [ ]:
#https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/
# split the data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, train_size = 0.8)

# scale data
standard_scaler = MinMaxScaler()
standard_scaler.fit(X_train)

X_train = standard_scaler.transform(X_train)
X_test = standard_scaler.transform(X_test)

print(np.shape(X_test))
print(np.shape(X_train))

Exibe a quantidade de letras em treino e em teste
Prepara-se as estruturas para montar o modelo, adequando à estrutura (\<letra\>, altura, largura, pixel)

In [ ]:
print(X_train.shape[0])
print(X_test.shape[0])
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype('float32')

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)



# Construindo e Treinando o modelo


Monta-se a rede neural com uma camada convolucional, uma de pooling

In [ ]:
cls = Sequential() # Define que a rede neural é linear
cls.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu')) # Layer convolucional
cls.add(MaxPooling2D(pool_size=(2, 2))) # Layer de pooling
cls.add(Dropout(0.3)) # Ignora parte dos neurônios
cls.add(Flatten()) # Transforma em um array
cls.add(Dense(128, activation='relu')) # layer que aumenta o tamanho da entrada do dado
cls.add(Dense(len(y.unique()), activation='softmax'))

cls.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Treina o modelo em 30 épocas com 2000 imagens em cada
history = cls.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=1200, verbose=2)

# Calcula a eficiência do modelo
scores = cls.evaluate(X_test,y_test, verbose=0)
print("CNN Score:",scores[1])

Demonstra a estatística do treino durante as épocas

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

Demonstra a matriz de confusão em que há os falsos negativos, falsos positivos e verdadeiros positivos

In [ ]:
cm=confusion_matrix(y_test.argmax(axis=1),cls.predict(X_test).argmax(axis=1))
df_cm = pd.DataFrame(cm, range(26),
                  range(26))
plt.figure(figsize = (20,15))
sns.set(font_scale=1.4)#for label size
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16})# font size

Salvando o modelo gerado

In [ ]:
cls.save('../modelos/modificado2.h5')